In [1]:
import pandas as pd
import os
from scipy import stats
from matplotlib.image import imread
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from matplotlib import rc
rc('font', family = 'AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [32]:
os.chdir('/home/kiho/바탕화면')

In [33]:
df= pd.read_csv("seoul_df.csv")

In [34]:
df_01 = df[df['reg_02.gs'] == '1분위수']

In [35]:
df_01 = df_01.drop(['date.gs','reg_02.gs'], axis = 1)
df_01.head(6)

,마스크,스타킹,스테디셀러,일회용품,"계절성(여름,겨울)",00~19,20~39,40~59,60~99,F,M,temp_max,wind_max,temp_min,temp_avg,rhumid_avg,wa_avg,rain_sum
0,77.0,77.0,8677.0,288.0,5071.0,357,9203,4293,337,8250,5940,4.8,2.6,-2.6,1.8,76.2,0.9,0.0
4,35.0,105.0,8617.0,267.0,4936.0,525,9456,3692,287,7622,6338,9.7,3.4,2.3,6.3,79.5,1.3,0.0
8,56.0,112.0,7528.0,266.0,4594.0,546,8535,3286,189,6988,5568,9.4,3.0,4.6,6.7,83.8,1.0,0.0
12,49.0,98.0,7314.0,337.0,4381.0,336,8124,3544,175,7392,4787,6.2,4.6,-1.6,2.8,54.5,2.3,0.0
16,21.0,56.0,8009.0,252.0,3534.0,434,8058,3205,175,6727,5145,1.6,4.1,-3.6,-1.6,41.5,1.8,0.0
20,14.0,119.0,9350.0,203.0,3443.0,533,9106,3238,252,6946,6183,2.3,3.5,-3.8,-0.8,57.5,1.3,0.0


In [18]:
data = list(df_01['마스크'])

In [42]:
from tensorflow.python.client import device_lib
import keras.backend.tensorflow_backend as K
#print(device_lib.list_local_devices())

# univariate cnn example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
np.random.seed(3)

In [43]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [44]:
n_steps = 3
# split into samples
X, y = split_sequence(data, n_steps)

In [48]:
# summarize the data
for i in range(10):
    print(X[i], y[i])

[[77.]
 [35.]
 [56.]] 49.0
[[35.]
 [56.]
 [49.]] 21.0
[[56.]
 [49.]
 [21.]] 14.0
[[49.]
 [21.]
 [14.]] 7.0
[[21.]
 [14.]
 [ 7.]] 14.0
[[14.]
 [ 7.]
 [14.]] 42.0
[[ 7.]
 [14.]
 [42.]] 35.0
[[14.]
 [42.]
 [35.]] 35.0
[[42.]
 [35.]
 [35.]] 70.0
[[35.]
 [35.]
 [70.]] 28.0


In [46]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [47]:
with K.tf.device('/gpu:0'): #'/cpu:0'
    # define model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, # kernel_siz는 filter랑 같음
                     activation='relu', # relu 불필요한걸 다 처낸다.
                     input_shape=(n_steps, n_features)))
    model.add(MaxPooling1D(pool_size=2)) # pool_size는 2~3
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # fit model
    model.fit(X, y, epochs=1000, verbose=0)

In [52]:
# demonstrate prediction
x_input = array([14, 42, 35])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print (yhat)

[[26.154034]]
